# Image-processing
### Shuffle images and then split 70% for training and 30% for validation

In [43]:
%%bash
shuf trial_02-ten_tags.lst > im.lst
image_count=`wc -l < im.lst | awk '{print int($0)}'`
# Trying to use image_count to automatically split 70/30
# (( image_count=image_count+1 ))
# (( train_count=image_count*(.7)))

head -n 3457 im.lst > train.lst
tail -n +3458 im.lst > val.lst
head train.lst
wc -l train.lst
wc -l val.lst

51628 	 0	0	1	0	0	0	0	0	0	0 	 /asset-image/user-2/node-51628/bolivia-star-silver-asset-shape-cutout-stamp-embellishment-gold-fall.png 
167959 	 1	0	0	0	0	1	0	0	1	0 	 /graphic-image/user-2725/node-167959/button-template-447-graphic-element-embellishment-fastener-sewing-vintage-antique-retro-shank-no.png 
40803 	 0	1	0	0	0	0	0	0	0	0 	 /asset-image/user-2/node-40803/mexico-leaf-007-asset-embellishment-leaves-extraction-green.png 
107009 	 0	0	0	0	0	0	0	1	0	0 	 /graphic-image/user-2/node-107009/birthday-print-word-circle1-graphic-tag-element-hybrid-untextured-printable-pink-white.png 
242269 	 0	0	0	0	0	0	0	1	0	0 	 /graphic-image/user-53886/node-242269/butterfly-spring-tag-graphic-scallop-ivory-brown-tan.png 
38307 	 0	0	0	0	0	1	0	0	0	1 	 /template-image/user-12831/node-38307/leafy-branch-illustration-template-09.png 
185867 	 0	0	0	0	0	0	1	1	0	0 	 /graphic-image/user-2725/node-185867/new-day-breathe-word-art-graphic-element-embellishment-snippet-tag-strip-navy-blue-purple.png 
20698 	 0	0

In [90]:
%%bash
[ "$(wc -l < train.lst)" -eq "$(sort -u train.lst | wc -l)" ] && echo all unique
[ "$(wc -l < val.lst)" -eq "$(sort -u val.lst | wc -l)" ] && echo all unique

all unique
all unique


In [97]:
import boto3
import botocore
import mxnet as mx
from PIL import Image
import numpy as np

lst = open('train.lst')
s3 = boto3.resource('s3')
images = []

bucket = 'pixelscrapper-user-content'
image_dir = 'images/'

train_rec = mx.recordio.MXRecordIO('train.rec', 'w')

train_rec.reset()

i = 0

for line in lst:
    
    line = line.split()
    s3_img_path = line[len(line) - 1][1:]
    local_img_path = s3_img_path.split('/')
    local_img_path = local_img_path[len(local_img_path) - 1]
    
    s3.Bucket(bucket).download_file(s3_img_path, image_dir + local_img_path)
    
    img_label = [float(i) for i in line[1:len(line)-1]] 
    img_id = int(line[0]) 
    img_header = mx.recordio.IRHeader(0, img_label, img_id, 0)
    
    img = Image.open(image_dir + local_img_path)
    img = img.convert("RGB")   
    img_array = np.array(img.resize((224, 224)))
    
    img_packed = mx.recordio.pack_img(img_header, img_array)
    train_rec.write(img_packed)
    
    os.remove(image_dir + local_img_path)
    
train_rec.close()
        


In [98]:
lst = open('val.lst')

bucket = 'pixelscrapper-user-content'
image_dir = 'images/'

val_rec = mx.recordio.MXRecordIO('val.rec', 'w')

val_rec.reset()

for line in lst:
    
    line = line.split()
    s3_img_path = line[len(line) - 1][1:]
    local_img_path = s3_img_path.split('/')
    local_img_path = local_img_path[len(local_img_path) - 1]
    
    s3.Bucket(bucket).download_file(s3_img_path, image_dir + local_img_path)
    
    img_label = [float(i) for i in line[1:len(line)-1]] 
    img_id = int(line[0]) 
    img_header = mx.recordio.IRHeader(0, img_label, img_id, 0)
    
    img = Image.open(image_dir + local_img_path)
    img = img.convert("RGB")   
    img_array = np.array(img.resize((224, 224)))
    
    img_packed = mx.recordio.pack_img(img_header, img_array)
    val_rec.write(img_packed)
    
    os.remove(image_dir + local_img_path)
    
val_rec.close()
        


In [99]:
bucket = 'sagemaker-multi-label-test'
prefix = 'ic-multilabel'

In [100]:
s3train = 's3://{}/{}/train/'.format(bucket, prefix)
s3validation = 's3://{}/{}/validation/'.format(bucket, prefix)

!aws s3 cp train.rec $s3train
!aws s3 cp val.rec $s3validation

upload: ./train.rec to s3://sagemaker-multi-label-test/ic-multilabel/train/train.rec
upload: ./val.rec to s3://sagemaker-multi-label-test/ic-multilabel/validation/val.rec
